# Homework Set 9

In [1]:
%pylab inline
import pandas as pd
import fmt

Populating the interactive namespace from numpy and matplotlib


## Problem 1


Similar to our example in class, here is the table of future liabilities (in $millions):

|  Years | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |
| :-----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: |
|  Benefits($millions) | 24 | 26 | 28 | 28 | 26 | 29 | 32 | 33 | 34 |


And here is the set of bonds that can be invested in:

|  Bonds | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 |
| :-----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: |
|  Price | 102 | 101.625 | 103 | 102 | 102 | 103 | 101 | 101.5 | 102 | 102.75 | 103 | 104 |
|  Coupon(%) | 1.25 | 1.5 | 2.5 | 1.75 | 2.125 | 2.5 | 2.625 | 3 | 3.125 | 3.5 |  3.75 | 4.5 |
|  Maturity(Years) | 1 | 2 | 2 | 3 | 4 | 5 | 5 | 6 | 7 | 8 |  8 | 9 |


Consider two excess cash scenarios: 1) not reused at each period. 2) reinvested at 1% interest rate and reused. 
Find the **least cost** portfolio of bonds so that the pension fund can meet its future liabilities. Please show your LP problem set up.


### Problem 1 Solution:

1) not resued at each period: <br>

Similar to the setup in lecture notebook, we could set up the LP problem as follows: <br>

\begin{aligned}
\min_x \, & \, (\sum_i x_iP_i) \\
\text{s.t. } \sum_{i = 1, \ldots, 12}\Big[(\mathbf{1}_{M_i \geq k}C_i(t) + \mathbf{1}_{M_i = k}\cdot 100)x_i\Big] \, & \, -s_k+s_{k-1} = L_k, \ k = 1,\ldots, 9 \\
x_i \geq 0, \, & \, i = 1,\ldots,12 \\
s_k \geq 0, \, & \, k = 1,\dots, 9
\end{aligned}

where: 

$\displaystyle L_k, k=1, \dots, 9$ are the future benefits (liabilities) payment at year $k$, $\displaystyle L = \begin{bmatrix} 24 & 26 & 28 & 28 & 26 & 29 & 32 & 33 & 34\end{bmatrix}$

$\displaystyle P_i, i=1, \dots, 12$ are the prices of bond $i$, $\displaystyle P = \begin{bmatrix} 102 & 101.625 & 103 & 102 & 102 & 103 & 101 & 101.5 & 102 & 102.75 & 103 & 104 \end{bmatrix}$

$\displaystyle C_i, i=1, \dots, 12$ are the coupons of bond $i$, $\displaystyle C = \begin{bmatrix} 1.25 & 1.5 & 2.5 & 1.75 & 2.125 & 2.5 & 2.625 & 3 & 3.125 & 3.5 & 3.75 & 4.5 \end{bmatrix}$

$\displaystyle M_i, i=1, \dots, 12$ are the maturity of bond $i$, $\displaystyle M = \begin{bmatrix} 1 & 2 & 2 & 3 & 4 & 5 & 5 & 6 & 7 & 8 & 8 & 9 \end{bmatrix}$

$\displaystyle x_i, i=1, \dots,12$ are the amount of bond $i$ to be purchased

$\displaystyle s_k, k=1, \dots, 9$ are the free cash at the end of year $k$

In [2]:
import numpy as np
from scipy.optimize import linprog

# Define arrays of future liabilities L, bonds price p, bonds coupon and bonds maturity
Coupon = np.array([1.25, 1.5, 2.5, 1.75, 2.125, 2.5, 2.625, 3, 3.125, 3.5, 3.75, 4.5])
Maturity = np.array([1, 2, 2, 3, 4, 5, 5, 6, 7, 8, 8, 9])

p = np.array([102., 101.625, 103., 102., 102., 103., 101., 101.5, 102, 102.75, 103., 104., 
             0, 0, 0, 0, 0, 0, 0, 0, 0])
L = np.array([24., 26., 28., 28., 26., 29., 32., 33., 34.])

# Define matrix of constraints for cash flow could not be reused
A1 = np.zeros((len(L), len(p)))
for i in range(len(L)):
    lenC = len(Coupon)
    for j in range(lenC):
        if (i <= Maturity[j] - 1):
            A1[i, j] += Coupon[j]
            if (i == Maturity[j] - 1):
                A1[i, j] += 100
    A1[i, i + lenC] = -1.
    if (i):
        A1[i, i + lenC - 1] = 1.

# solve LP and display the result
res1 = linprog(p, A_eq=A1, b_eq=L,
                options={ 'disp': True})
print(res1.x)

Bonds = ["Bond 1", "Bond 2", "Bond 3", "Bond 4", "Bond 5", "Bond 6", 
        "Bond 7", "Bond 8", "Bond 9", "Bond 10", "Bond 11", "Bond 12"]
fmt.displayDF(pd.DataFrame([res1.x[0:len(Coupon)]], columns = Bonds, 
                           index = ['The amount to be purchased: ']), fmt="4g")

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 1227.875            
0.289826388189      0.289826388189      0.289826388189      0.7234645774057  0.2898263881888     526.0511248698      
0.05888453529251    0.0588845352925     0.0588845352925     0.8180161648527  0.05888453529156    257.5159985632      
0.003198270909714   0.003198270901429   0.003198270901428   0.949073755942   0.003198269880027   226.4927295439      
3.798814444046e-06  3.798793902363e-06  3.798793902236e-06  0.9988790271259  3.797144395748e-06  225.1292438115      
1.206437872427e-10  1.209259759945e-10  1.209260676991e-10  0.9999681672442  1.898633809655e-10  225.1276886124      
Optimization terminated successfully.
         Current function value: 225.127689  
         Iterations: 5
[1.76159487e-01 2.15046709e-11 1.98361481e-01 2.23320518e-01
 2.2722

,Bond 1,Bond 2,Bond 3,Bond 4,Bond 5,Bond 6,Bond 7,Bond 8,Bond 9,Bond 10,Bond 11,Bond 12
The amount to be purchased:,0.1762,2.15e-11,0.1984,0.2233,0.2272,5.681e-12,0.2121,0.2476,0.2851,4.185e-11,0.304,0.3254


The least cost portfolio of bonds that excess cash could not be reused at each period, is showed above.

2) reinvested at 1% interest rate and reused

Using the same notation as we have defined in scenario 1), note that now reinvestment is allowed.

Then the free cash at the end of year $k - 1$, $s_{k - 1}$ would become $1.01 s_{k - 1}$ when we consider the cashflow in year $k$.

Hence the set up is as follows: 

\begin{aligned}
\min_x \, & \, (\sum_i x_iP_i) \\
\text{s.t. } \sum_{i = 1, \ldots, 12}\Big[(\mathbf{1}_{M_i \geq k}C_i(t) + \mathbf{1}_{M_i = k}\cdot 100)x_i\Big] \, & \, -s_k+1.01s_{k-1} = L_k, \ k = 1,\ldots, 9 \\
x_i \geq 0, \, & \, i = 1,\ldots,12 \\
s_k \geq 0, \, & \, k = 1,\dots, 9
\end{aligned}

In [3]:
# Define matrix of constraints for cash flow that could be reinvested at 1% interest rate and reused
A2 = np.zeros((len(L), len(p)))
for i in range(len(L)):
    lenC = len(Coupon)
    for j in range(lenC):
        if (i <= Maturity[j] - 1):
            A2[i, j] += Coupon[j]
            if (i == Maturity[j] - 1):
                A2[i, j] += 100
    A2[i, i + lenC] = -1.
    if (i):
        A2[i, i + lenC - 1] = 1.01

# solve LP and display the result
res2 = linprog(p, A_eq=A2, b_eq=L,
                options={ 'disp': True})

print(res2.x)

fmt.displayDF(pd.DataFrame([res2.x[0:len(Coupon)]], columns = Bonds, 
                           index = ['The amount to be purchased: ']), fmt="4g")

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 1227.875            
0.2856812482536     0.2856812482536     0.2856812482536     0.727444637814   0.2856812482535     525.1285643698      
0.05504862933328    0.05504862933327    0.05504862933327    0.8280143880858  0.05504862933313    256.3516800595      
0.003259098770232   0.00325909878595    0.00325909878595    0.944055471749   0.003259101060402   226.5627418177      
2.894665257858e-05  2.894664905442e-05  2.894664905418e-05  0.9914229659132  2.894661996677e-05  225.1395087105      
4.518432204902e-09  4.494211373691e-09  4.49421121961e-09   0.9998449447309  1.529541437403e-09  225.1276903907      
Optimization terminated successfully.
         Current function value: 225.127690  
         Iterations: 5
[1.76159489e-01 5.24653292e-11 1.98361503e-01 2.23320501e-01
 2.2722

,Bond 1,Bond 2,Bond 3,Bond 4,Bond 5,Bond 6,Bond 7,Bond 8,Bond 9,Bond 10,Bond 11,Bond 12
The amount to be purchased:,0.1762,5.247e-11,0.1984,0.2233,0.2272,1.985e-11,0.2121,0.2476,0.2851,1.253e-10,0.304,0.3254


The least cost portfolio of bonds that excess cash could be reinvested at 1% interest rate and reused, is showed above.